# Play with feature importance

Three questions to answer in here: 
    
(1) Is there anything interesting SHAP can tell use? 

(2) Is the importance the same for the surrogate models and the ones trained on a larger training set? The same for Gibbs and virial? 

(3) Does is change over the course of the PAL?

In [2]:
import sys 
import os 
from pathlib import Path 

%load_ext autoreload
%autoreload 2

import pandas as pd
import shap 

import matplotlib.pyplot as plt 
%matplotlib inline 

import seaborn as sns
plt.rcParams['font.family'] = 'sans-serif'

/Users/kevinmaikjablonka/opt/miniconda3/envs/dispersant_basf/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'shap'